# 1. Эксперимент #

## Блок нейтронной физики ##

In [1]:
import pandas as pd
import random as rd
import math as math
import scipy.constants as const # Модуль физико-математических констант

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

N_A = const.Avogadro

### Функция жизни нейтрона: ###

In [2]:
# Для одинаковой генерации данных
rd.seed(42)

def life_of_neutron(Ein, Eout, dens, A1, A2, Mol_m, Sigma, alpha_1, alpha_2, sigma_1, sigma_2, x1, x2):
    
    Energy = Ein
    colision = 0 # Число столкновений
    time = 0 # Время полета
    a = 0 # Угол полета
    b = 0 # Изменение угла полета
    l = 0 # Длина пролета
    V = 0 # Скорость
    elder = 0 #Возраст
    while Energy > Eout: #Пока энергия нейтрона лежит в области > ... ЭВ
        l = -1*math.log(rd.random())/Sigma/100 # В метрах
        V = (2*Energy*1.6*(10**(-19))/(1.6749*10**(-27)))**0.5 # В м/с
        time += l/V
        colision += 1
        
        #Разыгрывание на каком ядре произошло рассеяние
        x = rd.random()
        if x < (x1*sigma_1/(x1*sigma_1 + x2*sigma_2)):# Рассеяние на первом ядре
            E1 = Energy
            E2 = Energy*alpha_1
            Energy = rd.uniform(E1, E2)
            b = math.acos

            
        else:# Рассеяние на втором ядре
            E1 = Energy
            E2 = Energy*alpha_2
            Energy = rd.uniform(E1, E2)

    return colision, time

### Входные данные по материалам: ###

In [3]:
df = pd.DataFrame({'Material': ['Water', 'Heavy Plexiglass', 'Heavy Water'],
                   'Density_g/cm^3': [1.0, 1.6, 1.1],
                    'A_1' : [1, 12, 2],
                    'A_2' : [16, 2, 16],
                    'Molar_mass_g/mol' : [18, 28, 20],
                    'Chemical_formula' : ['H2O', 'C2D2', 'D2O'],
                    'sigma_1_b' : [30.0, 4.9, 3.5],
                    'sigma_2_b' : [4.0, 3.5, 4.0],
                    'x1' : [2, 2, 2],
                    'x2' : [1, 2, 1],
                    
              })

df

,Material,Density_g/cm^3,A_1,A_2,Molar_mass_g/mol,Chemical_formula,sigma_1_b,sigma_2_b,x1,x2
0,Water,1.0,1,16,18,H2O,30.0,4.0,2,1
1,Heavy Plexiglass,1.6,12,2,28,C2D2,4.9,3.5,2,2
2,Heavy Water,1.1,2,16,20,D2O,3.5,4.0,2,1


### Расчет макросечения: ###

In [4]:
df['\u03A3,_1/cm'] = 0.0
for k in df.index:
    df['Σ,_1/cm'][k] = (df['x1'][k]*df['sigma_1_b'][k]+df['x2'][k]*df['sigma_2_b'][k]
                       )*N_A*df['Density_g/cm^3'][k]/df['Molar_mass_g/mol'][k]/10**(24)
display(df)

,Material,Density_g/cm^3,A_1,A_2,Molar_mass_g/mol,Chemical_formula,sigma_1_b,sigma_2_b,x1,x2,"Σ,_1/cm"
0,Water,1.0,1,16,18,H2O,30.0,4.0,2,1,2.141206
1,Heavy Plexiglass,1.6,12,2,28,C2D2,4.9,3.5,2,2,0.578126
2,Heavy Water,1.1,2,16,20,D2O,3.5,4.0,2,1,0.364340


### Расчет ступеньки замедления: ###

In [5]:
df['\u0251_1'] = 0.0
df['\u0251_2'] = 0.0
for k in df.index:
    df['\u0251_1'][k] = ((df['A_1'][k]-1)/(df['A_1'][k]+1))**2
    df['\u0251_2'][k] = ((df['A_2'][k]-1)/(df['A_2'][k]+1))**2
display(df)

,Material,Density_g/cm^3,A_1,A_2,Molar_mass_g/mol,Chemical_formula,sigma_1_b,sigma_2_b,x1,x2,"Σ,_1/cm",ɑ_1,ɑ_2
0,Water,1.0,1,16,18,H2O,30.0,4.0,2,1,2.141206,0.000000,0.778547
1,Heavy Plexiglass,1.6,12,2,28,C2D2,4.9,3.5,2,2,0.578126,0.715976,0.111111
2,Heavy Water,1.1,2,16,20,D2O,3.5,4.0,2,1,0.364340,0.111111,0.778547


### Создание DataFrame для записи данных ###

In [6]:
df1 = pd.DataFrame({'Time': [],
                   'Colisions': [],
                    'Material' : [],
                    'Target' : [] 
              })
df1

,Time,Colisions,Material,Target


### Эксперимент методом Монте-Карло ###

In [7]:
Estart = 1_000_000 #1Мэв
Eend = 1 #1 Эв
Colision = [0, 0, 0, 0]
Time = [0, 0, 0, 0]
number_of_event = 300
counter = 0 # Счетчик для записи в DataFrame
for k in df.index: #Цикл по материалам
    for i in range(0, number_of_event*(k+1)): #Колличество событий для каждого материала
        colision, time = life_of_neutron(Estart, Eend, df['Density_g/cm^3'][k],
                                          df['A_1'][k], df['A_2'][k],
                                          df['Molar_mass_g/mol'][k], df['Σ,_1/cm'][k],
                                          df['ɑ_1'][k], df['ɑ_2'][k],
                                          df['sigma_1_b'][k], df['sigma_2_b'][k],
                                          df['x1'][k], df['x2'][k],)
        df1.loc[counter] = {'Material': df['Material'][k], 'Time': time, 'Colisions': colision, 'Target': k}
        Colision[k]+=colision
        Time[k]+=time
        counter+=1


for k in df.index:
    Colision[k] = Colision[k]/number_of_event/(k+1)
    Time[k] = Time[k]/number_of_event/(k+1)
    
for k in df.index:
    print(df['Material'][k])
    print(Colision[k])
    print(Time[k])

Water
15.643333333333333
6.787573276044247e-07
Heavy Plexiglass
35.90833333333333
6.268212392367504e-06
Heavy Water
28.513333333333335
7.811704616661977e-06


# 2. SQL vs Pandas

## 2.1 Подключение к данным, CREATE TABLE, INSERT, выгрузка данных

### 2.1.1 Подключение к данным Pandas

In [8]:
df

,Material,Density_g/cm^3,A_1,A_2,Molar_mass_g/mol,Chemical_formula,sigma_1_b,sigma_2_b,x1,x2,"Σ,_1/cm",ɑ_1,ɑ_2
0,Water,1.0,1,16,18,H2O,30.0,4.0,2,1,2.141206,0.000000,0.778547
1,Heavy Plexiglass,1.6,12,2,28,C2D2,4.9,3.5,2,2,0.578126,0.715976,0.111111
2,Heavy Water,1.1,2,16,20,D2O,3.5,4.0,2,1,0.364340,0.111111,0.778547


In [9]:
df1

,Time,Colisions,Material,Target
0,6.013426e-07,18.0,Water,0.0
1,1.526499e-07,11.0,Water,0.0
2,5.879935e-07,17.0,Water,0.0
3,4.702673e-07,14.0,Water,0.0
4,1.814087e-07,10.0,Water,0.0
...,...,...,...,...
1795,1.073471e-05,37.0,Heavy Water,2.0
1796,4.577775e-06,28.0,Heavy Water,2.0
1797,1.017680e-05,35.0,Heavy Water,2.0
1798,5.321271e-06,29.0,Heavy Water,2.0


### 2.1.2 Подключение к данным SQLite

#### 2.1.2.1 DF to Data Base SQLite

In [10]:
import sqlite3

conn = sqlite3.connect('data/neutron_moderation_SQLite.db')
c = conn.cursor()

df.to_sql('Input_data', conn, if_exists='replace', index = False)
df1.to_sql('Output_data', conn, if_exists='replace', index = False)

#### 2.1.2.2 Data Base SQLite to DF

In [11]:
df = pd.read_sql("SELECT * FROM Input_data", conn)
df

,Material,Density_g/cm^3,A_1,A_2,Molar_mass_g/mol,Chemical_formula,sigma_1_b,sigma_2_b,x1,x2,"Σ,_1/cm",ɑ_1,ɑ_2
0,Water,1.0,1,16,18,H2O,30.0,4.0,2,1,2.141206,0.000000,0.778547
1,Heavy Plexiglass,1.6,12,2,28,C2D2,4.9,3.5,2,2,0.578126,0.715976,0.111111
2,Heavy Water,1.1,2,16,20,D2O,3.5,4.0,2,1,0.364340,0.111111,0.778547


In [12]:
df1 = pd.read_sql("SELECT * FROM Output_data", conn)
df1

,Time,Colisions,Material,Target
0,6.013426e-07,18.0,Water,0.0
1,1.526499e-07,11.0,Water,0.0
2,5.879935e-07,17.0,Water,0.0
3,4.702673e-07,14.0,Water,0.0
4,1.814087e-07,10.0,Water,0.0
...,...,...,...,...
1795,1.073471e-05,37.0,Heavy Water,2.0
1796,4.577775e-06,28.0,Heavy Water,2.0
1797,1.017680e-05,35.0,Heavy Water,2.0
1798,5.321271e-06,29.0,Heavy Water,2.0


In [13]:
c.close()
conn.close()

### 2.1.3 Подключение к данным MySQL

#### 2.1.3.1 DF to Data Base MySQL

In [14]:
import pymysql

#### 2.1.3.2 Data Base MySQL to DF

### 2.1.4 Подключение к данным MS SQL Server

#### 2.1.4.1 DF to Data Base MS SQL Server

In [15]:
#Для корректной работы необходимо создать базу данных через MS SQL Server Management Studio
# имя базы [neutron_moderation_MS_SQL_Server]

#Подключимся к созданной таблице и выведем данные

import pyodbc
import pandas as pd

server = 'IVANPC' 
database = 'neutron_moderation_MS_SQL_Server' 
username = 'sa' 
password = 'sasa'

Установка и настройка MS SQL Server и Management Studio

https://www.youtube.com/watch?v=dP_ZmYhNFlg&ab_channel=%D0%92%D0%B8%D0%B4%D0%B5%D0%BE%D1%83%D1%80%D0%BE%D0%BA%D0%B8%D0%BF%D0%BE%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8EALEKSEEV74

In [16]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [17]:
#Удалить таблицу если такая уже существует

query = '''
DROP TABLE IF EXISTS Output_data
'''
cursor.execute(query)
cnxn.commit()

In [18]:
#создание таблицы Output_data

query = '''
CREATE TABLE [dbo].[Output_data](
	[Time] [float] NOT NULL,
	[Colisions] [int] NOT NULL,
	[Material] [varchar](50) NOT NULL,
	[Target] [int] NOT NULL,
	[Ind] [int] NOT NULL
)
'''
cursor.execute(query)
cnxn.commit()

In [19]:
#ограничение на уникальность ключа

query = '''
ALTER TABLE Output_data
ADD CONSTRAINT DF_Output_data_Ind_Unique UNIQUE (Ind)
'''
cursor.execute(query)
cnxn.commit()

In [20]:
#установление Primary Key

query = '''
ALTER TABLE Output_data
ADD CONSTRAINT PK_Output_data_index  PRIMARY KEY CLUSTERED (Ind)
'''
cursor.execute(query)
cnxn.commit()

In [21]:
#Показать созданную таблицу

query = "SELECT * FROM Output_data;"
df1_from_MS_SQL_Server = pd.read_sql(query, cnxn)
df1_from_MS_SQL_Server

,Time,Colisions,Material,Target,Ind


In [22]:
#Запись данных

cursor = cnxn.cursor()

for index,row in df1.iterrows():
    cursor.execute('''
        INSERT INTO Output_data(
        [Time], [Colisions],[Material],[Target],[Ind]
        ) 
        values (?, ?,?,?,?)''',
                   row['Time'], 
                   row['Colisions'],
                   row['Material'],
                   row['Target'],
                   index
                  ) 
cnxn.commit()

In [23]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [24]:
#Удалить таблицу если такая уже существует

query = '''
DROP TABLE IF EXISTS Input_data
'''
cursor.execute(query)
cnxn.commit()

In [25]:
#создание таблицы Input_data

query = '''
CREATE TABLE [dbo].[Input_data](
	[Material] [varchar](50) NOT NULL,
	[Density_g/cm^3] [float] NOT NULL,
	[A_1] [int] NOT NULL,
	[A_2] [int] NOT NULL,
	[Molar_mass_g/mol] [varchar](50) NOT NULL,
	[Chemical_formula] [varchar](50) NOT NULL,
	[sigma_1_b] [float] NOT NULL,
	[sigma_2_b] [float] NOT NULL,
	[x1] [int] NOT NULL,
	[x2] [int] NOT NULL,
	[Σ,_1/cm] [float] NOT NULL,
	[ɑ_1] [float] NOT NULL,
	[ɑ_2] [float] NOT NULL
)
'''
cursor.execute(query)
cnxn.commit()

In [26]:
#ограничение на уникальность ключа

query = '''
ALTER TABLE Input_data
ADD CONSTRAINT DF_Input_data_data_Ind_Unique UNIQUE (Material)
'''
cursor.execute(query)
cnxn.commit()

In [27]:
#установление Primary Key

query = '''
ALTER TABLE Input_data
ADD CONSTRAINT PK_Input_data_Material  PRIMARY KEY CLUSTERED (Material)
'''
cursor.execute(query)
cnxn.commit()

In [28]:
#Показать созданную таблицу

query = "SELECT * FROM Input_data;"
df_from_MS_SQL_Server = pd.read_sql(query, cnxn)
df_from_MS_SQL_Server

,Material,Density_g/cm^3,A_1,A_2,Molar_mass_g/mol,Chemical_formula,sigma_1_b,sigma_2_b,x1,x2,"Σ,_1/cm",ɑ_1,ɑ_2


In [29]:
#Запись данных

cursor = cnxn.cursor()

for index,row in df.iterrows():
    cursor.execute('''
        INSERT INTO Input_data(
        [Material],
        [Density_g/cm^3],
        [A_1],
        [A_2],
        [Molar_mass_g/mol],
        [Chemical_formula],
        [sigma_1_b],
        [sigma_2_b],
        [x1],
        [x2],
        [Σ,_1/cm],
        [ɑ_1],
        [ɑ_2]
        )
        values (?,?,?,?,?,?,?,?,?,?,?,?,?)''',
        row['Material'],
        row['Density_g/cm^3'],
        row['A_1'],
        row['A_2'],
        row['Molar_mass_g/mol'],
        row['Chemical_formula'],
        row['sigma_1_b'],
        row['sigma_2_b'],
        row['x1'],
        row['x2'],
        row['Σ,_1/cm'],
        row['ɑ_1'],
        row['ɑ_2']
) 
cnxn.commit()

#### 2.1.4.2 Data Base MS SQL Server to DF

In [30]:
# Выгрузка данных

query = "SELECT * FROM Output_data;"
df1_from_MS_SQL_Server = pd.read_sql(query, cnxn)
df1_from_MS_SQL_Server

,Time,Colisions,Material,Target,Ind
0,6.013426e-07,18,Water,0,0
1,1.526499e-07,11,Water,0,1
2,5.879935e-07,17,Water,0,2
3,4.702673e-07,14,Water,0,3
4,1.814087e-07,10,Water,0,4
...,...,...,...,...,...
1795,1.073471e-05,37,Heavy Water,2,1795
1796,4.577775e-06,28,Heavy Water,2,1796
1797,1.017680e-05,35,Heavy Water,2,1797
1798,5.321271e-06,29,Heavy Water,2,1798


In [31]:
# Выгрузка данных

query = "SELECT * FROM Input_data;"
df_from_MS_SQL_Server = pd.read_sql(query, cnxn)
df_from_MS_SQL_Server

,Material,Density_g/cm^3,A_1,A_2,Molar_mass_g/mol,Chemical_formula,sigma_1_b,sigma_2_b,x1,x2,"Σ,_1/cm",ɑ_1,ɑ_2
0,Heavy Plexiglass,1.6,12,2,28,C2D2,4.9,3.5,2,2,0.578126,0.715976,0.111111
1,Heavy Water,1.1,2,16,20,D2O,3.5,4.0,2,1,0.364340,0.111111,0.778547
2,Water,1.0,1,16,18,H2O,30.0,4.0,2,1,2.141206,0.000000,0.778547


In [32]:
cursor.close()
cnxn.close()

## 2.2 Query

### 2.2.1 Query Pandas

In [33]:
df1.groupby(by=["Material"]).mean()[['Time', 'Colisions']].sort_values(by='Colisions', ascending=False)

,Time,Colisions
Material,,
Heavy Plexiglass,6.268212e-06,35.908333
Heavy Water,7.811705e-06,28.513333
Water,6.787573e-07,15.643333


In [34]:
# Проверим медиану
df1.groupby(by=["Material"]).median()[['Time', 'Colisions']].sort_values(by='Colisions', ascending=False)

,Time,Colisions
Material,,
Heavy Plexiglass,5.763001e-06,35.0
Heavy Water,6.893299e-06,28.0
Water,5.343644e-07,16.0


### 2.2.1 Query SQLite

In [35]:
conn = sqlite3.connect('data/neutron_moderation_SQLite.db')
c = conn.cursor()

query = '''
SELECT [Material],
	AVG([Time]) as AVG_Time,
	AVG([Colisions]) as AVG_Colisions
FROM Output_data
GROUP BY [Material]
ORDER BY AVG_Colisions DESC
'''

Query_from_SQLite = pd.read_sql(query, conn)
Query_from_SQLite

,Material,AVG_Time,AVG_Colisions
0,Heavy Plexiglass,6.268212e-06,35.908333
1,Heavy Water,7.811705e-06,28.513333
2,Water,6.787573e-07,15.643333


In [36]:
c.close()
conn.close()

### 2.2.3 Query MySQL

### 2.2.4 Query MS SQL Server

In [37]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

query = '''
SELECT [Material],
	AVG([Time]) as AVG_Time,
	AVG([Colisions]) as AVG_Colisions
FROM Output_data
GROUP BY [Material]
ORDER BY AVG_Colisions DESC
'''
Query_from_MS_SQL_Server = pd.read_sql(query, cnxn)
Query_from_MS_SQL_Server

,Material,AVG_Time,AVG_Colisions
0,Heavy Plexiglass,6.268212e-06,35
1,Heavy Water,7.811705e-06,28
2,Water,6.787573e-07,15


In [38]:
cursor.close()
cnxn.close()

Происходит округление, так как Colisions - это int в базе MS SQL Server, это мы задали сами. Отличается от Pandas и SQLite

In [39]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Time       1800 non-null   float64
 1   Colisions  1800 non-null   float64
 2   Material   1800 non-null   object 
 3   Target     1800 non-null   float64
dtypes: float64(3), object(1)
memory usage: 56.4+ KB


## 2.3 Subquery

### 2.3.1 Subquery Pandas

In [40]:
df_subquery = df1.groupby(by=["Material"]).mean()[['Time', 'Colisions']]
df_subquery = df_subquery.loc[df_subquery['Colisions'] > 20]
df_subquery

,Time,Colisions
Material,,
Heavy Plexiglass,0.000006,35.908333
Heavy Water,0.000008,28.513333


### 2.3.2 Subquery SQLite

In [41]:
conn = sqlite3.connect('data/neutron_moderation_SQLite.db')
c = conn.cursor()

query = '''
SELECT *
FROM (
    SELECT [Material],
	AVG([Time]) as [AVG_Time],
	AVG([Colisions]) as [AVG_Colisions]
    FROM [Output_data]
    GROUP BY [Material]) as q1
WHERE [AVG_Colisions] > 20
'''

Query_from_SQLite = pd.read_sql(query, conn)
Query_from_SQLite

,Material,AVG_Time,AVG_Colisions
0,Heavy Plexiglass,0.000006,35.908333
1,Heavy Water,0.000008,28.513333


In [42]:
c.close()
conn.close()

### 2.3.3 Subquery MySQL

### 2.3.4 Subquery MS SQL Server

In [43]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

query = '''
SELECT *
FROM (
    SELECT [Material],
	AVG([Time]) as [AVG_Time],
	AVG([Colisions]) as [AVG_Colisions]
    FROM [Output_data]
    GROUP BY [Material]) as q1
WHERE [AVG_Colisions] > 20
'''
Query_from_MS_SQL_Server = pd.read_sql(query, cnxn)
Query_from_MS_SQL_Server

,Material,AVG_Time,AVG_Colisions
0,Heavy Plexiglass,0.000006,35
1,Heavy Water,0.000008,28


## 2.4 Join and CTE

### 2.4.1 Join and CTE Pandas

In [44]:
CTE = pd.merge(df, df1, how='left', on='Material')
CTE = CTE[['Material', 'Time', 'Colisions']]
CTE = CTE[CTE.Material.isin(['Water', 'Heavy Water'])]
CTE

,Material,Time,Colisions
0,Water,6.013426e-07,18.0
1,Water,1.526499e-07,11.0
2,Water,5.879935e-07,17.0
3,Water,4.702673e-07,14.0
4,Water,1.814087e-07,10.0
...,...,...,...
1795,Heavy Water,1.073471e-05,37.0
1796,Heavy Water,4.577775e-06,28.0
1797,Heavy Water,1.017680e-05,35.0
1798,Heavy Water,5.321271e-06,29.0


### 2.4.2 Join and CTE SQLite

### 2.4.2 Join and CTE MySQL

### 2.4.3 Join and CTE MS SQLServer